In [ ]:
# Tell the notebook where to find the pipeline functions
import sys
sys.path.append('/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Lesion_Pipeline/')

In [ ]:
import os
import shutil
from glob import glob 
import GNMpipeline as p

# Add your inputs

In [ ]:
input_dir="/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Pipeline/RDCRN_test_output/"  #Folder with input files, can use the folder created by Combine script previously
input_type='BIDS' #'BIDS' or 'Folder'
output_dir="/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Pipeline/RDCRN_test_output/"


subjects_to_skip=None #['c002', 'c003'] 

skullstrip='synthstrip' #'synthstrip', 'optibet', or None 


IMAGE_TYPES = ['t1', 'T2', 'FLAIR'] #case sensitive, change to match what you used e.g. t1, t1w, TI 
Reg_target_1='t1' #your ideal registration target, case sensitive
Reg_target_2='T2'

# Make dictionary of subjects and sessions

In [ ]:
subject_sessions=p.create_input_dict(input_dir, subjects_to_skip=subjects_to_skip, input_type=input_type)

# Co-register images to registration target and apply skullstripping if desired

In [ ]:
if skullstrip not in ['optibet', 'synthstrip', None]:
    raise ValueError("Please change your skullstrip variable to either 'optibet', 'synthstrip', or None")

co_reg_dict={}
for subject, sessions in subject_sessions.items():
    reg_target=None
    subject_folder = os.path.join(output_dir, subject) 
    for session in sessions:
        print(f'*** Processing {subject}: {session} ***')
        session_folder=os.path.join(subject_folder, session)
        
        reg_target=p.set_registration_target(glob(f'{session_folder}/SELECTED/*.nii*'), Reg_target_1, Reg_target_2)
        print(reg_target)
        
        print(f'Registering Images to the {reg_target}') #~5 min per file
        
        if not os.path.exists(f'{session_folder}/COREGISTERED'):
            os.makedirs(f'{session_folder}/COREGISTERED')
            
        reg_file=f'{session_folder}/SELECTED/{subject}_{session}_{reg_target}.nii.gz'
        shutil.copy(reg_file, f'{session_folder}/COREGISTERED')
        
        #Get brain mask to apply skullstripping to other images once they are co-registered 
        brain_mask=None
        if skullstrip != None:
            suffix = '_' + skullstrip
            brain_mask=f'{session_folder}/SELECTED/*brain-mask*{suffix}*'
            shutil.copy(f'{session_folder}/SELECTED/{subject}_{session}_{reg_target}_SkullStripped{suffix}.nii.gz', f'{session_folder}/COREGISTERED/{subject}_{session}_{reg_target}_SkullStripped.nii.gz')
        
        for image in IMAGE_TYPES:
            if image != reg_target:
                
                file=f'{session_folder}/SELECTED/{subject}_{session}_{image}.nii.gz'

                if os.path.exists(file):
                    shutil.copy(file, f'{session_folder}/COREGISTERED')

                    print(f'Registering {file}')
                    command=p.co_register(f'{session_folder}/COREGISTERED', reg_file, file, tag=f'space-{reg_target}', brain_mask=brain_mask, clean_up=True)
                    job_name = f"co-register_{subject}_{session}_{reg_target}"
                    job_id=p.submit_slurm_job(job_name, command)
                    co_reg_dict[(subject,session,os.path.basename(file))] = job_id
                else:
                    print(f'Warning: {file} does not exist')

if len(co_reg_dict) > 0:
    print('You have', len(co_reg_dict), 'Co-Registration Jobs submitted to SLURM; subject and job IDs are stored in the co_reg_dict')
    print('You can type "squeue -u $USER" into your terminal  or "!squeue -u $USER" in a cell to track SLURM job progress')
    print('You can check the output file matching the jobid in co_reg_dict to see code outputs and any errors')
                                        

# Check your submitted jobs

In [ ]:
print("These are the jobs that should have been submitted:")
print(co_reg_dict)

In [ ]:
!squeue -u $USER 